In [ ]:
from IPython.display import display, HTML
from sympy.abc import alpha
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
import re

# Set projects, LCI and LCIA methods

In [ ]:
BW_PROJECT = 'regioinvent_2025' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

In [ ]:
EI_DB = 'ecoinvent-3.10-cutoff'
RI_DB = 'Regioinvent'
LIB_RM_DB = 'LIB raw materials'

In [ ]:
for act in bw.Database("LIB raw materials"):
    print(act)

## LCI

### Search for keywords in EI and export information on preselected LCI

In [ ]:
from useful_lca_functions import filter_ecoinvent_activities

In [ ]:
# Define filters
databases = ['ecoinvent-3.10-cutoff']
products = ['aluminium', 'aluminum', 'cadmium', 'copper', 'diamond', 'gold',
            'niobium', 'nickel', 'palladium', 'platinum', 'potash',
            'tellurium', 'titanium', 'uranium']
#locations = None  # Set to None to include all locations OR provide a list like ['CA', 'US', 'DE']

df_lci_ei310 = filter_ecoinvent_activities(databases, products, locations_to_include=None)
df_lci_ei310_can = filter_ecoinvent_activities(databases, products, locations_to_include=['CA', 'CA-QC'])

In [ ]:
df_lci_ei310.to_csv(r'review/ei310_lci.csv', index=False)
df_lci_ei310_can.to_csv(r'review/ei310_can_lci.csv', index=False)

### Picking the chosen LCI

In [ ]:
# (mineral name: activity name, reference product, location)
INVENTORIES_CA = {
    "Aluminium primary ingot": ('aluminium production, primary, ingot',
                                'aluminium, primary, ingot',
                                'CA'),

    "Aluminium primary liquid": ("aluminium production, primary, liquid, prebake", 
                                "aluminium, primary, liquid", 
                                "CA"),

    "Cadmium": ('cadmium production, primary', 
                'cadmium', 
                'CA-QC'),
    
    "Copper concentrate CA": ('copper mine operation and beneficiation, sulfide ore', 
                           'copper concentrate, sulfide ore', 
                           'CA'),
    
    "Copper concentrate CA (gold)": ('gold-silver mine operation and beneficiation', 
                                  'copper concentrate, sulfide ore', 
                                  'CA-QC'),
    
    "Gold production" : ("gold production", 
                         "gold", 
                         "CA"),
    
    "Gold mining and beneficiation": ("gold-silver mine operation and beneficiation", 
                                      "gold-silver, ingot", 
                                      "CA-QC"),
    
    "Gold operation refinery": ("gold-silver mine operation with refinery",
                                "gold", 
                                'CA-QC'),    
    
    "Molybdenite CA": ('copper mine operation and beneficiation, sulfide ore', 
                    'molybdenite', 
                    'CA'),
    
    "Nickel concentrate CA": ('nickel mine operation and benefication to nickel concentrate, 16% Ni',
                           'nickel concentrate, 16% Ni',
                           'CA-QC'),
    
    "Sulfidic tailings copper CA": ('treatment of sulfidic tailings, from copper mine operation, tailings impoundment',
                                 'sulfidic tailings, from copper mine operation', 
                                 'CA'),
    
    "Sulfidic market tailings gold CA": ("market for sulfidic tailings, from gold mine operation", 
                                  "sulfidic tailings, from gold mine operation", 
                                  "CA"),
    
    "Sulfidic tailings gold CA": ("treatment of sulfidic tailings, from gold mine operation, tailings impoundment",
                                  "sulfidic tailings, from gold mine operation", 
                                  "CA"),
    
    "Sulfidic tailings nickel CA": ('treatment of sulfidic tailings, from nickel mine operation, tailings impoundment',
                           'sulfidic tailings, from nickel mine operation',
                           'CA'),
    
    "Tellurium": ("tellurium production, semiconductor-grade",
                  "tellurium, semiconductor-grade", 
                  "CA-QC")
}
    

In [ ]:
    # From Istrate et al (2024)
    #"Nickel ore mining": ('nickel ore mining, average excluding China',
    #                      'nickel ore, mined',
    #                      'GLO'),
    #"Nickel concentrate": ('nickel concentration, average excluding China',
    #                       'nickel concentrate',
    #                       'GLO'),
    #"Nickel matte": ('nickel matte production, nickel-cobalt sulphite and nickel sub materials, mass allocation',
    #                 'nickel matte',
    #                 'GLO') # relevant for smelting? intermediate product obtained from smelting nickel concentrate

In [ ]:
# # (mineral name: activity name, reference product, location)
# INVENTORIES = {
#     ## From EI
#     "Neodymium":        ("rare earth oxides production, from rare earth carbonate concentrate", "neodymium oxide", "RoW"),
#     "Copper concentrate, Canada": ('copper mine operation and beneficiation, sulfide ore', 'copper concentrate, sulfide ore', 'CA'),
#     "Market for copper, cathode": ('market for copper, cathode', 'copper, cathode', 'GLO'),
#     
#     # From Istrate et al (2024)
#     "Lithium hydroxide, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
#     "Lithium hydroxide, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
#     "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
#     "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
#     "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
#     "Graphite, synthetic":   ("graphite powder coating", "synthetic graphite, battery grade", "CN"),
#     
#     "Lithium carbonate, brine":        ("lithium carbonate production, Salar de Atacama", "lithium carbonate, battery grade", "CL"),
#     "Lithium carbonate, spodumene":    ("lithium carbonate production, from Australian spodumene", "lithium carbonate, battery grade", "CN"),
# }

In [ ]:
# INVENTORIES_EI = {
#     "Neodymium": ("market for neodymium oxide", "neodymium oxide", "GLO"), # CAN not here
#     "Copper": ("market for copper, cathode", "copper, cathode", 'GLO'),    
#     "Lithium carbonate": ("market for lithium carbonate", "lithium carbonate", "GLO"),
#     "Cobalt oxide": ("market for cobalt oxide", "cobalt oxide", "GLO"),
#     "Nickel": ("market for nickel, class 1", "nickel, class 1", "GLO"),
#     "Graphite": ("market for graphite", "graphite", "GLO")
# }

In [ ]:
# # If we take consumption market activities 
# # (mineral name: activity name, reference product, location)
# INVENTORIES_RI = {
#     "Neodymium": ("consumption market for neodymium oxide", "neodymium oxide", "US"), # CAN not here
#     "Copper": ("consumption market for copper, cathode", "copper, cathode", 'CA'),    
#     "Lithium carbonate": ("consumption market for lithium carbonate", "lithium carbonate", "CA"),
#     "Cobalt oxide": ("consumption market for cobalt oxide", "cobalt oxide", "CA"),
#     "Nickel": ("consumption market for nickel, class 1", "nickel, class 1", "CA"),
#     "Graphite": ("consumption market for graphite", "graphite", "US"), # US     
# }

In [ ]:
#INVENTORIES_EI_ds = get_inventory_dataset(INVENTORIES_EI, database_names=[EI_DB, LIB_RM_DB])
#INVENTORIES_RI_ds = get_inventory_dataset(INVENTORIES_RI, database_names=[RI_DB])

In [ ]:
from useful_lca_functions import get_inventory_dataset

In [ ]:
INVENTORIES_CA_ds = get_inventory_dataset(INVENTORIES_CA, database_names=[EI_DB])

## Import IW+2.1 LCIA methods

In [ ]:
# We can also import some from IW+2.1
expert_ei310 = 'data/IW+2.1/impact_world_plus_21_brightway2_expert_version_ei310.5535d12bedce3770ffef004e84229fd1.bw2package'
bw.BW2Package.import_file(expert_ei310)

In [ ]:
# Filter and display metho|ds that contain "IMPACT World+" in their names
IW_METHODS = [method for method in bd.methods if "impact world+" in " ".join(method).lower()]
df_iw_methods = pd.DataFrame(IW_METHODS, columns=["Method", "Impact Category", "Subcategory"])
df_iw_methods.to_csv(r'data/IW+2.1/iw_methods_3.10.csv', index=False)

In [ ]:
# To take them all
IMPACT_METHODS_ALL = {method[-1]: method for method in IW_METHODS}

In [ ]:
# Extract methods for IMPACT World+ Damage 2.1
IW_DAMAGE = [
    method for method in IW_METHODS 
    if method[0] == 'IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10'
]

In [ ]:
# Extract methods for IMPACT World+ Midpoint 2.1
IW_MIDPOINTS = [
    method for method in IW_METHODS 
    if method[0] == 'IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10'
]

In [ ]:
IW_EP = {
'Total HH': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Human health', 'Total human health'), 
'Total EQ': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Ecosystem quality', 'Total ecosystem quality'), 
}


# Calculate specific impacts (e.g. per mass)

## LCA calculations

In [ ]:
from useful_lca_functions import multi_lcia, run_multi_lcia_for_lcis

In [ ]:
df_ep = run_multi_lcia_for_lcis(INVENTORIES_CA_ds, amount=1, lcia_methods=IW_EP)

In [ ]:
df_ep

In [ ]:
df_ep.to_csv(r'results/link_with_metallican/df_ep.csv', index=False)

In [ ]:
from visualisation_functions import plot_multilca_impacts

In [ ]:
fig_lca_results = plot_multilca_impacts(df_ep, 
                 colors=["#ff7f0e", "#2ca02c"], 
                 save_path="results/specific_lca_results/lca_impacts_ei.png")

## Midpoints contribution on EP

In [ ]:
df_damage = run_multi_lcia_for_lcis(INVENTORIES_CA_ds, amount=1, lcia_methods=IW_DAMAGE)

In [ ]:
df_damage

In [ ]:
df_damage.to_csv(r'results/link_with_metallican/df_damage.csv', index=False)

In [ ]:
from visualisation_functions import plot_iwplus_contributions

In [ ]:
plot_iwplus_contributions(df_damage, save_path_eco="results/link_with_metallican/ecosystem_quality_contributions.png", save_path_hh="results/link_with_metallican/human_health_contributions.png")

## Contribution analysis

In [ ]:
from useful_lca_functions import contribution_analysis_technosphere

In [ ]:
df_contribution = contribution_analysis_technosphere(INVENTORIES_CA_ds, amount=1, lcia_methods=IW_EP)

In [ ]:
df_contribution

In [ ]:
df_contribution.to_csv(r'results/link_with_metallican/df_contribution.csv', index=False)

In [ ]:
from visualisation_functions import pie_charts_technosphere_contribution

In [ ]:
pie_charts_technosphere_contribution(df_contribution, legend_size=12, percentage_threshold=5, save_path='results/link_with_metallican')